In [1]:
import pandas as pd
import numpy as np

In [21]:
import datetime

In [4]:
df_ride=pd.read_csv('ride.csv')

In [5]:
df_ride.head()

,id_request,id_driver,rating,actual_revenue,created_at
0,c34d4f3600e,1ef5333d,NaN,3.00,2018-03-01T00:01:11.000Z
1,e8435acac43,e4577c05,NaN,0.00,2018-03-01T00:02:41.000Z
2,329b0a67f8f,413107f5,NaN,2.62,2018-03-01T00:05:52.000Z
3,3d83fd431bb,1fb445d0,5.0,3.25,2018-03-01T00:06:27.000Z
4,dd758cc2c52,238395d4,NaN,7.62,2018-03-01T00:14:51.000Z


In [6]:
df_request=pd.read_csv('request.csv')

In [7]:
df_request.head()

,id_request,latitude,longitude,has_promo,created_at
0,853bdb5ae5d,-12.113449,-76.995270,False,2018-03-01T00:01:40.000Z
1,589462c80bb,-12.063645,-77.051477,False,2018-03-01T00:02:00.000Z
2,64c2538f0ad,-12.305687,-76.840968,False,2018-03-01T00:03:45.000Z
3,185cce2998d,-12.083725,-77.025562,False,2018-03-01T00:03:59.000Z
4,b1519ed99c0,-12.105328,-76.970738,False,2018-03-01T00:07:23.000Z


In [10]:
df_fare=pd.read_csv('fare_snapshot.csv')

C:\Users\n_zizos\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df_fare.head()

,id_passenger,id_request,estimate_fare,estimate_distance,estimate_duration
0,9b25c99e77,NaN,14.5,11.56,1735
1,86bf28afdc,NaN,47.0,24.00,4398
2,cef4ba758a,NaN,26.0,22.54,2567
3,23c1c474de,NaN,9.5,5.19,901
4,d70f356592,NaN,26.0,15.46,1277


In [15]:
df_fare_2=df_fare.dropna()

In [22]:
len(df_fare)

11521480

In [25]:
len(df_fare)-len(df_fare.dropna())

6500339

In [24]:
len(df_request)

5021141

In [37]:
len(df_ride[df_ride['rating'].isnull()==True])/len(df_ride)

0.8037650778403054

In [40]:
df_ride[df_ride['created_at'].isnull()==True]

,id_request,id_driver,rating,actual_revenue,created_at


In [9]:
len(df_ride)

3730972

In [16]:
### Adding passenger to ride
df_ride_2=pd.merge(df_ride,df_fare_2[['id_request','id_passenger']])

In [17]:
df_ride_2.head()

,id_request,id_driver,rating,actual_revenue,created_at,id_passenger
0,c34d4f3600e,1ef5333d,NaN,3.00,2018-03-01T00:01:11.000Z,347f9e5abf
1,e8435acac43,e4577c05,NaN,0.00,2018-03-01T00:02:41.000Z,e932431e16
2,329b0a67f8f,413107f5,NaN,2.62,2018-03-01T00:05:52.000Z,d521789996
3,3d83fd431bb,1fb445d0,5.0,3.25,2018-03-01T00:06:27.000Z,ed4caef1b3
4,dd758cc2c52,238395d4,NaN,7.62,2018-03-01T00:14:51.000Z,d94dd17c3d


In [23]:
#adding date

df_ride_2['date']=df_ride_2['created_at'].apply(lambda x: pd.to_datetime(x))

In [39]:
df_ride_2['date']=df_ride_2['date'].apply(lambda x: x.date())

In [40]:

df_ride_2.head()

,id_request,id_driver,rating,actual_revenue,created_at,id_passenger,date
0,c34d4f3600e,1ef5333d,NaN,3.00,2018-03-01T00:01:11.000Z,347f9e5abf,2018-03-01
1,e8435acac43,e4577c05,NaN,0.00,2018-03-01T00:02:41.000Z,e932431e16,2018-03-01
2,329b0a67f8f,413107f5,NaN,2.62,2018-03-01T00:05:52.000Z,d521789996,2018-03-01
3,3d83fd431bb,1fb445d0,5.0,3.25,2018-03-01T00:06:27.000Z,ed4caef1b3,2018-03-01
4,dd758cc2c52,238395d4,NaN,7.62,2018-03-01T00:14:51.000Z,d94dd17c3d,2018-03-01


## RFM Calculation

In [96]:
##Filter dates up to April

df_ride_3=df_ride_2[df_ride_2['date']<=datetime.date(2018,3,31)]

In [97]:
max_date=max(df_ride_3['date'])

In [98]:
max_date

datetime.date(2018, 3, 31)

In [99]:
rfmTable = df_ride_3.groupby('id_passenger').agg({'date': lambda x: (max_date - x.max()).days, # Recency
                                        'id_request': lambda x: len(x),      # Frequency
                                        'actual_revenue': lambda x: x.sum()}) # Monetary

In [100]:
rfmTable['date'] = rfmTable['date'].astype(int)
rfmTable.rename(columns={'date': 'recency', 
                         'id_request': 'frequency', 
                         'actual_revenue': 'monetary_value'}, inplace=True)

In [101]:
rfmTable.head()

,recency,frequency,monetary_value
id_passenger,,,
000008ac3b,0,5,18.86
000010ece4,9,3,3.00
00001b49cf,6,9,26.61
00001c4c35,14,2,3.99
0000443808,10,2,3.37


## RFM Quartiles

In [102]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [103]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,2.0,4.87
0.50,5.0,4.0,12.12
0.75,13.0,10.0,28.61


In [104]:
quantiles = quantiles.to_dict()

In [105]:
quantiles

{'recency': {0.25: 2.0, 0.5: 5.0, 0.75: 13.0},
 'frequency': {0.25: 2.0, 0.5: 4.0, 0.75: 10.0},
 'monetary_value': {0.25: 4.869999885559082,
  0.5: 12.119999885559082,
  0.75: 28.609999656677246}}

In [106]:

rfmSegmentation = rfmTable


In [107]:

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)# Argume 
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [108]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [109]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [110]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id_passenger,,,,,,,
000008ac3b,0,5,18.86,1,2,2,122
000010ece4,9,3,3.00,3,3,4,334
00001b49cf,6,9,26.61,3,2,2,322
00001c4c35,14,2,3.99,4,4,4,444
0000443808,10,2,3.37,3,4,4,344


In [111]:
### Top 5 customers

In [112]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)


,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id_passenger,,,,,,,
7c4305c0bf,0,302,812.779992,1,1,1,111
c5865d86b4,0,120,534.819996,1,1,1,111
4ed4c33a62,0,156,492.519995,1,1,1,111
a1bd29092b,0,168,474.039993,1,1,1,111
517f5b187b,0,97,470.699996,1,1,1,111


In [113]:
df_ride_3[df_ride_3['id_passenger']=='7c4305c0bf']

,id_request,id_driver,rating,actual_revenue,created_at,id_passenger,date
9601,901a4809232,c2ae585a,NaN,1.87,2018-03-16T15:45:24.000Z,7c4305c0bf,2018-03-16
18241,5cdfd8bef59,04a29a5d,NaN,1.75,2018-03-30T13:56:42.000Z,7c4305c0bf,2018-03-30
21945,9540bdb581d,fe7286bc,NaN,1.75,2018-03-05T20:27:21.000Z,7c4305c0bf,2018-03-05
53467,ab86dcb043e,1bcb8f6f,NaN,1.87,2018-03-25T21:28:48.000Z,7c4305c0bf,2018-03-25
72774,806c402964c,b2f90c80,NaN,3.37,2018-03-26T15:45:25.000Z,7c4305c0bf,2018-03-26
74890,a875b699b0a,24853b95,4.0,2.75,2018-03-30T02:12:16.000Z,7c4305c0bf,2018-03-30
94099,8c6a6033486,68f35089,4.0,2.62,2018-03-30T19:43:04.000Z,7c4305c0bf,2018-03-30
121496,1c389d06d32,7517eb4b,4.0,1.75,2018-03-14T02:08:29.000Z,7c4305c0bf,2018-03-14
122348,fe2a1efdfe9,0fb9c564,NaN,3.00,2018-03-15T15:40:03.000Z,7c4305c0bf,2018-03-15
122793,de8ed7db19f,d099cf98,4.0,1.75,2018-03-16T03:50:12.000Z,7c4305c0bf,2018-03-16


In [117]:
rfmSegmentation.to_csv('rfmSegmentation_raw.csv')